In [143]:
import import_hack
import steps
from data_store import sqlite_table_datastore as sqlds
from data_store import file_system_directory_datastore as fsds
from data_store import stream_ndarray_adapter_datastore as snads
from data_store import numpy_datastore as npds
from transformer import transformers as trs
from search import inverted_multi_index_searcher as imis
from quantization import pq_quantizer
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [206]:
productsiftbincount_sqlite_db_path='ds_data\imgid_productsiftbincount'
productsiftbincount_sqlite_table_name='imgid_productsiftbincount'
productsiftbincount_clusters_sqlite_db_path='ds_data\imgid_productsiftbincount_clusters'
productsiftbincount_clusters_sqlite_table_name='clusterid_cluster'

In [189]:
# productsiftbincount_sqlite_db_path='ds_data\imgid_productsiftbincount_L2normalized'
# productsiftbincount_sqlite_table_name='imgid_productsiftbincountL2normalized'
# productsiftbincount_clusters_sqlite_db_path='ds_data\imgid_productsiftbincountL2normalized_clusters'
# productsiftbincount_clusters_sqlite_table_name='clusterid_cluster'

In [207]:
productsiftbincount_ds = sqlds.SQLiteTableDataStore(productsiftbincount_sqlite_db_path, productsiftbincount_sqlite_table_name)
productsiftbincount_clusters_ds = sqlds.SQLiteTableDataStore(productsiftbincount_clusters_sqlite_db_path, productsiftbincount_clusters_sqlite_table_name)

In [208]:
cluster_centers_ndarray_ds=snads.StreamNdarrayAdapterDataStore(productsiftbincount_clusters_ds, detect_final_shape_by_first_elem=True)
cluster_centers=cluster_centers_ndarray_ds.get_items_sorted_by_ids()

In [209]:
with productsiftbincount_ds:
    items=productsiftbincount_ds.get_items_sorted_by_ids()
    for i in range(1):
        item=next(items)
        print(item.shape)
#         print(item)

(600,)


In [210]:
print(cluster_centers.shape)
# for i in range(2):
#     print(cluster_centers[i])

(2, 250, 300)


In [211]:
productsiftbincount_ndarray_ds = snads.StreamNdarrayAdapterDataStore(productsiftbincount_ds, detect_final_shape_by_first_elem=True)
X = productsiftbincount_ndarray_ds.get_items_sorted_by_ids()
X_ids=productsiftbincount_ndarray_ds.get_ids_sorted()
print(X.shape)
print(X_ids.shape)

(999, 600)
(999,)


In [212]:
searcher_=imis.InvertedMultiIndexSearcher(X, X_ids, cluster_centers)

In [213]:
np_neighbor_ids_ds=npds.NumpyDataStore()
steps.search_step(productsiftbincount_ds, searcher_, 6 , np_neighbor_ids_ds)

In [214]:
np_neighbor_ids_ds.get_items_sorted_by_ids()

array([[  1,   8,   4,   9,   6,   5],
       [  9,   6,   5,   3,   2,   8],
       [  9,   6,   5,   3,   2,   8],
       ..., 
       [998, 997, 994, 991, 995, 999],
       [998, 997, 994, 991, 995, 999],
       [999, 996, 992, 993, 995,  90]])